This file grabs everything from the LSMS survey that I think an image could possibly recognize and uses those features to predict consumption. This serves as a "gold standard" for any image-based model. It turns out that the CNN model performs almost as well as this gold standard!

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
BASE_DIR = '..'
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data/Nightlights/2013/F182013.v4c_web.stable_lights.avg_vis.tif')
COUNTRY = 'malawi_2016'

LSMS_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY, 'LSMS')
PROCESSED_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY, 'processed')

In [3]:
# these vary from one LSMS survey to another
CONSUMPTION_FILE = 'IHS4 Consumption Aggregate.dta'
CONSUMPTION_PH_COL = 'rexpagg' # per household
CONSUMPTION_PC_COL = 'rexpaggpc' # per capita

GEOLOCATION_FILE = 'HouseholdGeovariables_stata11/HouseholdGeovariablesIHS4.dta'
LATITUDE_COL = 'lat_modified'
LONGITUDE_COL = 'lon_modified'

# purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
PPP = 207.238

In [4]:
for file in [CONSUMPTION_FILE, GEOLOCATION_FILE]:
    assert os.path.isfile(os.path.join(LSMS_DIR, file)), print(f'Could not find {file}')

In [5]:
df_geo = pd.read_stata(os.path.join(LSMS_DIR, GEOLOCATION_FILE))
df_hhf = pd.read_stata(os.path.join(LSMS_DIR, 'HH_MOD_F.dta'))
df_plot = pd.read_stata(os.path.join(LSMS_DIR, 'PlotGeovariablesIHS4.dta'))

In [6]:
df_com = pd.read_stata(os.path.join(LSMS_DIR, 'COM_CD.dta'))
df_com2 = pd.read_stata(os.path.join(LSMS_DIR, 'COM_CF1.dta'))

In [7]:
# rooms = df_hhf['hh_f10']
# roof = df_hhf['hh_f08']

# # all distance infrasturcture metrics
# road_type = df_com['com_cd01']
# dist_daily_market = df_com['com_cd16']
# dist_larger_weekly = df_com['com_cd18a']
# dist_perm_admarc = df_com['com_cd20a']
# dist_post_office = df_com['com_cd22a']
# dist_telephone = df_com['com_cd24a']
# dist_gov_prim_school = df_com['com_cd27a']
# dist_gov_sec_school = df_com['com_cd36a']
# dist_comm_sec_school = df_com['com_cd40a']
# dist_medicines = df_com['com_cd49a']
# dist_health_clinic = df_com['com_cd51a']
# dist_doctor = df_com['com_cd60a']
# dist_bank = df_com['com_cd67a']
# dist_microfinance = df_com['com_cd69a']

# dist_agric_exten_officer = df_com2['com_cf08a']

# dist_admarc_outlet = df_geo['dist_admarc']
# dist_agric_market = df_geo['dist_agmrkt']
# dist_tobacco_auction = df_geo['dist_auction']
# dist_boma = df_geo['dist_boma']
# dist_border = df_geo['dist_borderpost']
# dist_popcenter = df_geo['dist_popcenter']
# dist_road = df_geo['dist_road']

# dist_hh = df_plot['dist_hh']

# # temp
# mean_temp = df_geo['af_bio_1']
# mean_temp_wet_q = df_geo['af_bio_8']

# # rain
# mean_rain = df_geo['af_bio_12']
# mean_rain_wet_month = df_geo['af_bio_13']
# mean_rain_wet_q = df_geo['af_bio_16']

In [8]:
# for the purpose of merging dfs with case_ids and ea_ids together
df_tie = pd.read_stata(os.path.join(LSMS_DIR, CONSUMPTION_FILE))[['case_id', 'ea_id']]

hhf_input = df_hhf[['case_id', 'hh_f10', 'hh_f08']]
com_input = df_com[['ea_id', 'com_cd01', 'com_cd16', 'com_cd18a', 'com_cd20a', 'com_cd22a', 'com_cd24a',
                   'com_cd27a', 'com_cd36a', 'com_cd40a', 'com_cd49a', 'com_cd51a', 'com_cd60a', 'com_cd67a',
                   'com_cd69a']]

com2_input = df_com2[['ea_id', 'com_cf08a']]

geo_input = df_geo[['case_id', 'dist_admarc', 'dist_agmrkt', 'dist_auction', 'dist_boma', 'dist_borderpost',
                  'dist_popcenter', 'dist_road', 'af_bio_1', 'af_bio_8', 'af_bio_12', 'af_bio_13', 'af_bio_16', 
                   'lat_modified', 'lon_modified']]
geo_input.rename(columns={'lat_modified': 'cluster_lat', 'lon_modified': 'cluster_lon'}, inplace=True)
geo_input.dropna(inplace=True)

plot_input = df_plot[['case_id', 'dist_hh']]

/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [9]:
df_cons = pd.read_csv(os.path.join(PROCESSED_DIR, 'clusters.csv'))
df_cons.rename(columns={'lat': 'cluster_lat', 'lon': 'cluster_lon'}, inplace=True)
df_cons.head()

,cluster_lat,cluster_lon,cons_pc,nightlights
0,-17.095150,35.217213,1.477796,0.000000
1,-17.092351,35.114643,1.314741,0.000000
2,-17.016698,35.079629,1.626932,0.000000
3,-16.977243,35.205706,1.733232,0.121212
4,-16.956385,35.168967,1.131669,0.502674


In [10]:
def merge_on_lat_lon(df1, df2, keys=['cluster_lat', 'cluster_lon']):
    """
        Allows two dataframes to be merged on lat/lon
        Necessary because pandas has trouble merging on floats
    """
    df1 = df1.copy()
    df2 = df2.copy()
    
    # must use ints for merging, as floats induce errors
    df1['merge_lat'] = (10000 * df1[keys[0]]).astype(int)
    df1['merge_lon'] = (10000 * df1[keys[1]]).astype(int)
    
    df2['merge_lat'] = (10000 * df2[keys[0]]).astype(int)
    df2['merge_lon'] = (10000 * df2[keys[1]]).astype(int)
    
    df2.drop(keys, axis=1, inplace=True)
    merged = pd.merge(df1, df2, on=['merge_lat', 'merge_lon'])
    merged.drop(['merge_lat', 'merge_lon'], axis=1, inplace=True)
    return merged

In [11]:
df_merge = merge_on_lat_lon(df_cons, geo_input)
df_merge = pd.merge(df_merge, hhf_input, on='case_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, df_tie, on='case_id', how='left')
df_merge = pd.merge(df_merge, com_input, on='ea_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, com2_input, on='ea_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, plot_input, on='case_id', how='left')
print(df_merge.shape)

(12444, 19)
(12444, 34)
(12444, 35)
(19865, 36)


In [12]:
df_merge.head()

,cluster_lat,cluster_lon,cons_pc,nightlights,case_id,dist_admarc,dist_agmrkt,dist_auction,dist_boma,dist_borderpost,...,com_cd27a,com_cd36a,com_cd40a,com_cd49a,com_cd51a,com_cd60a,com_cd67a,com_cd69a,com_cf08a,dist_hh
0,-17.09515,35.217213,1.477796,0.0,311017590042,1.0,21.0,145.0,21.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2
1,-17.09515,35.217213,1.477796,0.0,311017590010,2.0,20.0,145.0,20.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-17.09515,35.217213,1.477796,0.0,311017590064,2.0,20.0,145.0,20.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7
3,-17.09515,35.217213,1.477796,0.0,311017590064,2.0,20.0,145.0,20.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7
4,-17.09515,35.217213,1.477796,0.0,311017590146,2.0,20.0,145.0,20.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9


In [13]:
df_merge.columns

Index(['cluster_lat', 'cluster_lon', 'cons_pc', 'nightlights', 'case_id',
       'dist_admarc', 'dist_agmrkt', 'dist_auction', 'dist_boma',
       'dist_borderpost', 'dist_popcenter', 'dist_road', 'af_bio_1',
       'af_bio_8', 'af_bio_12', 'af_bio_13', 'af_bio_16', 'hh_f10', 'hh_f08',
       'ea_id', 'com_cd01', 'com_cd16', 'com_cd18a', 'com_cd20a', 'com_cd22a',
       'com_cd24a', 'com_cd27a', 'com_cd36a', 'com_cd40a', 'com_cd49a',
       'com_cd51a', 'com_cd60a', 'com_cd67a', 'com_cd69a', 'com_cf08a',
       'dist_hh'],
      dtype='object')

In [14]:
df_final = df_merge

In [15]:
df_use = df_final.drop(['case_id', 'ea_id'], axis=1)

In [16]:
df_use = pd.get_dummies(df_use)

In [17]:
clusters = df_use.groupby(['cluster_lat', 'cluster_lon'])

In [18]:
cluster_df = clusters.mean().reset_index()

In [19]:
cluster_df

,cluster_lat,cluster_lon,cons_pc,nightlights,dist_admarc,dist_agmrkt,dist_auction,dist_boma,dist_borderpost,dist_popcenter,...,hh_f08_GRASS,hh_f08_IRON SHEETS,hh_f08_CLAY TILES,hh_f08_CONCRETE,hh_f08_PLASTIC SHEETING,hh_f08_OTHER (SPECIFY),com_cd01_TAR/ASPHALT,com_cd01_GRADED GRAVELED,com_cd01_DIRT ROAD (MAINTAINED),com_cd01_DIRT TRACK
0,-17.095150,35.217213,1.477796,0.000000,1.500000,20.125000,145.000000,20.125000,4.125000,20.125000,...,0.666667,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-17.092351,35.114643,1.314741,0.000000,8.105263,25.578947,146.368421,25.578947,10.105263,25.578947,...,0.789474,0.210526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-17.016698,35.079629,1.626932,0.000000,15.761905,23.047619,134.857143,23.047619,21.523810,23.047619,...,0.571429,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-16.977243,35.205706,1.733232,0.121212,6.970588,11.764706,135.764706,11.764706,13.500000,11.764706,...,0.705882,0.294118,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-16.956385,35.168967,1.131669,0.502674,13.000000,13.681818,130.181818,13.681818,20.636364,13.681818,...,0.454545,0.545455,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,-9.591378,33.057450,1.463979,0.000000,7.666667,26.222222,235.277778,26.222222,5.944444,103.666667,...,0.444444,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
776,-9.550397,33.291558,1.290441,0.000000,10.185185,18.370370,228.740741,18.370370,17.481481,82.481481,...,0.407407,0.592593,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
777,-9.519230,33.139193,1.873278,0.000000,5.057143,24.971429,238.400000,24.971429,17.428571,98.828571,...,0.371429,0.628571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
778,-9.507538,33.259649,1.860406,0.000000,4.465116,21.604651,234.209302,21.604651,18.441860,90.000000,...,0.255814,0.744186,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# a few columns have a high percentage of NA
nas = cluster_df.isna().sum() / len(cluster_df)
nas[nas > 0]

com_cd16     0.379487
com_cd18a    0.350000
com_cd20a    0.257692
com_cd22a    0.151282
com_cd24a    0.191026
com_cd27a    0.032051
com_cd36a    0.032051
com_cd40a    0.032051
com_cd49a    0.603846
com_cd51a    0.288462
com_cd60a    0.032051
com_cd67a    0.078205
com_cd69a    0.214103
com_cf08a    0.434615
dist_hh      0.042308
dtype: float64

# Modeling

In [21]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns


def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1


In [22]:
def nan_handler(df):
    nas = df.isna().sum()
    for c in df:
        if nas[c] > 0:
            df[c] = df[c].fillna(df[c].median())
    return df

In [23]:
cleaned_df = nan_handler(cluster_df)

In [24]:
cleaned_df.head()

,cluster_lat,cluster_lon,cons_pc,nightlights,dist_admarc,dist_agmrkt,dist_auction,dist_boma,dist_borderpost,dist_popcenter,...,hh_f08_GRASS,hh_f08_IRON SHEETS,hh_f08_CLAY TILES,hh_f08_CONCRETE,hh_f08_PLASTIC SHEETING,hh_f08_OTHER (SPECIFY),com_cd01_TAR/ASPHALT,com_cd01_GRADED GRAVELED,com_cd01_DIRT ROAD (MAINTAINED),com_cd01_DIRT TRACK
0,-17.095150,35.217213,1.477796,0.000000,1.500000,20.125000,145.000000,20.125000,4.125000,20.125000,...,0.666667,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-17.092351,35.114643,1.314741,0.000000,8.105263,25.578947,146.368421,25.578947,10.105263,25.578947,...,0.789474,0.210526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-17.016698,35.079629,1.626932,0.000000,15.761905,23.047619,134.857143,23.047619,21.523810,23.047619,...,0.571429,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-16.977243,35.205706,1.733232,0.121212,6.970588,11.764706,135.764706,11.764706,13.500000,11.764706,...,0.705882,0.294118,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-16.956385,35.168967,1.131669,0.502674,13.000000,13.681818,130.181818,13.681818,20.636364,13.681818,...,0.454545,0.545455,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
y = cleaned_df['cons_pc'].values
y_log = np.log(y)

to_drop = ['cluster_lat', 'cluster_lon', 'cons_pc']
x = cleaned_df.drop(to_drop, axis=1).values


In [26]:
# better than the CNN, but not by that much
_, _, _, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 27.825594022071243
best alpha 10.0


0.5413365285501057

In [27]:
_, _, _, r2 = predict_consumption(x, y)
r2

best alpha 10.0
best alpha 77.4263682681127
best alpha 10.0
best alpha 10.0
best alpha 10.0


0.3672653722387111

In [28]:
def create_train_valid(x, y, frac=0.7):
    n_train = int(0.7*(len(x)))
    inds = np.arange(len(x))
    train_ind = np.random.choice(inds, n_train, replace=False)
    valid_ind = np.delete(inds, train_ind)

    train_x = x[train_ind]
    valid_x = x[valid_ind]

    train_y = y[train_ind]
    valid_y = y[valid_ind]

    ss = StandardScaler() # standardize features
    train_x = ss.fit_transform(train_x)
    valid_x = ss.transform(valid_x)
    
    return train_x, train_y, valid_x, valid_y

In [29]:
train_x, train_y, valid_x, valid_y = create_train_valid(x, y_log, frac=0.7)

In [30]:
ridge = linear_model.Ridge(alpha=20)
ridge.fit(train_x, train_y)
ridge.score(train_x, train_y)

0.6384134135446173

In [31]:
ridge.score(valid_x, valid_y)

0.5101590918741898

In [32]:
ridge.intercept_

0.7498751782134738

In [33]:
ridge.coef_

array([ 0.07075874, -0.00383365,  0.01945366, -0.02573349, -0.01393802,
        0.03479431, -0.0185782 , -0.02215639,  0.00779353, -0.06012495,
       -0.13196374,  0.0003342 ,  0.07391796, -0.02725232,  0.01962565,
       -0.00448584, -0.00953821,  0.04505238,  0.02820707,  0.00412629,
        0.03305999,  0.00616236, -0.01258239, -0.02131538,  0.03104118,
        0.02072928, -0.06307757,  0.00663819,  0.03695454, -0.1045175 ,
        0.10478933,  0.00438912, -0.00259713, -0.02479541,  0.02256877,
        0.0156085 , -0.02588725, -0.05815515, -0.01267294])

In [34]:
df_imps = pd.DataFrame.from_dict({'columns': cleaned_df.drop(to_drop,axis=1).columns,
                                 'imps': ridge.coef_})
df_imps.sort_values('imps', ascending=False, inplace=True)

In [35]:
df_imps.head(10)

,columns,imps
30,hh_f08_IRON SHEETS,0.104789
12,af_bio_16,0.073918
0,nightlights,0.070759
17,com_cd22a,0.045052
28,dist_hh,0.036955
5,dist_borderpost,0.034794
20,com_cd36a,0.033060
24,com_cd60a,0.031041
18,com_cd24a,0.028207
34,hh_f08_OTHER (SPECIFY),0.022569


In [36]:
df_imps.tail(10)

,columns,imps
7,dist_road,-0.022156
33,hh_f08_PLASTIC SHEETING,-0.024795
3,dist_auction,-0.025733
36,com_cd01_GRADED GRAVELED,-0.025887
13,hh_f10,-0.027252
37,com_cd01_DIRT ROAD (MAINTAINED),-0.058155
9,af_bio_8,-0.060125
26,com_cd69a,-0.063078
29,hh_f08_GRASS,-0.104517
10,af_bio_12,-0.131964
